Stage 4, Report
https://github.com/anhaidgroup/py_entitymatching/blob/master/notebooks/vldb_demo/Demo_notebook_v6.ipynb

In [24]:
import py_entitymatching as em
import os
import pandas as pd

# specify filepaths for tables A and B. 
path_A = 'tableA.csv'
path_B = 'tableB.csv'
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='id')
# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B, key='id')
# Load the pre-labeled data
S = em.read_csv_metadata('sample_labeled.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
# Split S into I an J
IJ = em.split_train_test(S, train_proportion=0.75, random_state=3)
I = IJ['train']
J = IJ['test']
print(A.columns.values)
print(B.columns.values)
print('-------------------')
#S.head()

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


['id' 'name' 'year' 'rating' 'rank' 'min_num_players' 'max_num_players'
 'min_gameplay_time' 'max_gameplay_time' 'min_age' 'complexity_weight'
 'category' 'mechanisms' 'type' 'BGG_link' 'store_names' 'store_prices'
 'links_to_buy' 'availability' 'international_store']
['id' 'name' 'year' 'rating' 'rank' 'num_players' 'min_num_players'
 'max_num_players' 'gameplay_time' 'min_gameplay_time' 'max_gameplay_time'
 'min_age' 'complexity_weight' 'category' 'mechanisms' 'type' 'BGG_link'
 'store_names' 'store_prices' 'links_to_buy' 'availability'
 'international_store']
-------------------


In [25]:
# Generate a set of features
F = em.get_features_for_matching(A, B)
print(F)

# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, feature_table=F, attrs_after='label', show_progress=False)
H.head()
H.axes

Column BGG_link does not seem to qualify as any atomic type. It may contain all NaNs. Currently, setting its type to be un_determined.We recommend the users to manually update the returned types or features later. 

Column store_names does not seem to qualify as any atomic type. It may contain all NaNs. Currently, setting its type to be un_determined.We recommend the users to manually update the returned types or features later. 

Column store_prices does not seem to qualify as any atomic type. It may contain all NaNs. Currently, setting its type to be un_determined.We recommend the users to manually update the returned types or features later. 

Column links_to_buy does not seem to qualify as any atomic type. It may contain all NaNs. Currently, setting its type to be un_determined.We recommend the users to manually update the returned types or features later. 

Column availability does not seem to qualify as any atomic type. It may contain all NaNs. Currently, setting its type to be u

                                    feature_name     left_attribute  \
0                                      id_id_exm                 id   
1                                      id_id_anm                 id   
2                                 id_id_lev_dist                 id   
3                                  id_id_lev_sim                 id   
4                                  year_year_exm               year   
5                                  year_year_anm               year   
6                             year_year_lev_dist               year   
7                              year_year_lev_sim               year   
8            min_num_players_min_num_players_exm    min_num_players   
9            min_num_players_min_num_players_anm    min_num_players   
10      min_num_players_min_num_players_lev_dist    min_num_players   
11       min_num_players_min_num_players_lev_sim    min_num_players   
12           max_num_players_max_num_players_exm    max_num_players   
13    

[Int64Index([ 75,  14,  47,  46, 379, 159,  79, 265,  30, 209,
             ...
             169, 363, 202, 147, 277, 256, 131, 249, 152, 362],
            dtype='int64', length=300),
 Index(['_id', 'ltable_id', 'rtable_id', 'id_id_exm', 'id_id_anm',
        'id_id_lev_dist', 'id_id_lev_sim', 'year_year_exm', 'year_year_anm',
        'year_year_lev_dist', 'year_year_lev_sim',
        'min_num_players_min_num_players_exm',
        'min_num_players_min_num_players_anm',
        'min_num_players_min_num_players_lev_dist',
        'min_num_players_min_num_players_lev_sim',
        'max_num_players_max_num_players_exm',
        'max_num_players_max_num_players_anm',
        'max_num_players_max_num_players_lev_dist',
        'max_num_players_max_num_players_lev_sim',
        'min_gameplay_time_min_gameplay_time_exm',
        'min_gameplay_time_min_gameplay_time_anm',
        'min_gameplay_time_min_gameplay_time_lev_dist',
        'min_gameplay_time_min_gameplay_time_lev_sim',
        'max_g

In [26]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [28]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 
                       'ltable_id', 
                       'rtable_id', 
                       'id_id_exm', 
                       'id_id_anm',
        'id_id_lev_dist', 
                       'id_id_lev_sim', 
                       'year_year_exm', 
                       'year_year_anm',
        'year_year_lev_dist', 
                       'year_year_lev_sim',
        'min_num_players_min_num_players_exm',
        'min_num_players_min_num_players_anm',
        #'min_num_players_min_num_players_lev_dist',
        'min_num_players_min_num_players_lev_sim',
        'max_num_players_max_num_players_exm',
        'max_num_players_max_num_players_anm',
        #'max_num_players_max_num_players_lev_dist',
        'max_num_players_max_num_players_lev_sim',
        'min_gameplay_time_min_gameplay_time_exm',
        'min_gameplay_time_min_gameplay_time_anm',
        #'min_gameplay_time_min_gameplay_time_lev_dist',
        'min_gameplay_time_min_gameplay_time_lev_sim',
        'max_gameplay_time_max_gameplay_time_exm',
        'max_gameplay_time_max_gameplay_time_anm',
        #'max_gameplay_time_max_gameplay_time_lev_dist',
        'max_gameplay_time_max_gameplay_time_lev_sim', 
        'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Decision Tree

In [ ]:


# Check whether the updated F improves X (Random Forest)
result = em.select_matcher([rf], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

##  Random Forest

## SVM

## Naive Bayes

## Logistic Regression

For each of the five learning methods (Decision Tree, Random Forest, SVM, Naive Bayes, Logistic Regression), 
Report the precision
Recall
F-1 
that you obtain when you perform cross validation for the first time for these methods on I.

Report which learning based matcher you selected after that cross validation.

Report all debugging iterations and cross validation iterations that you performed. For each debugging
iteration, report (a) what is the matcher that you are trying to debug, and its precision/recall/F-1, (b)
what kind of problems you found, and what you did to fix them, (c) the final precision/recall/F-1 that
you reached

For each cross validation iteration, report (a) what matchers were you trying to evaluate using the
cross validation, and (b) precision/recall/F-1 of those.

• Report the final best learning-based matcher that you selected, and its precision/recall/F-1.

 Now report the following:
– For each of the five learning methods, train it on I, then report its precision/recall/F-1 on J.
– For the final best matcher Y∗,  train it on I then report its precision/recall/F-1 on J
- List the final set of features that you are using in your feature vectors. 

• Report an approximate time estimate: (a) how much did it take to label the data, and (b) to find the best
learning-based matcher.

• Discuss why you can't reach higher precision, recall, F-1. 